# Operators

Liste des opérateurs de la base

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import networkx as nx
import datetime
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Chargement des variables du .env
load_dotenv()
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
sys.path.append('/notebooks/recipes')

# fallback to .env connection string
try: connection_string
except NameError: connection_string = os.environ['PG_CONNECTION_STRING']

# connect to DB
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
with engine.connect() as conn: df_result = pd.read_sql_query(text("SELECT current_timestamp AT TIME ZONE 'Europe/Paris'"), conn)
print('Notebook initialised at: ' + str(df_result.values[0][0]))


In [ ]:
from itables import show

query = f"""
  SELECT _id, upper(uuid::text) as UUID, name, legal_name, siret, cgu_accepted_at, created_at
  FROM operator.operators
  WHERE deleted_at IS NULL
  ORDER BY name
"""

with engine.connect() as conn: df_result = pd.read_sql_query(text(query), conn)

df_result['idx'] = df_result.index
show(df_result)
